# 第2单元: 使用FrozenLake-v1 ⛄️ 和Taxi-v3 🚕 进行 Q-Learning
在这份笔记中, **你将从头开始编写你的第一个强化学习智能体**使用 Q-Learning 来玩FrozenLake ❄️ 并将其分享到社区, 并记得实验不同的配置

❓如果你有任何问题, 请在discord的#study-group-unit2频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境: 
* [FrozenLake-v1](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/)
* [Taxi-v3](https://www.gymlibrary.ml/environments/toy_text/taxi/)

📚 强化学习库: Python和NumPy

⬇️ 这是**你将在几分钟内实现的目标**的示例. ⬇️

![envs.gif](./assets/envs.gif)

## 这份笔记的目标🏆

在这份笔记学习结束后, 你将:

* 能够使用环境库**Gym**.
* 能够从头开始编写一个Q-Learning智能体.
* 能够通过精彩的回放和得分🔥**发布你训练的智能体到Hugging Face Hub**.

## 这份笔记来自深度强化学习课程
![深度强化学习课程.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践**.
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ 在独特的环境中训练**智能体**.

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第2单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit2/README.md)

🔲 📚 [阅读**Q-Learning简介第1部分**](https://huggingface.co/blog/deep-rl-q-part1)

🔲 📚 [阅读**Q-Learning简介第2部分**](https://huggingface.co/blog/deep-rl-q-part2)

🔲 📢 注册我们的Discord服务器并在#introduce-yourself频道介绍自己 🥳

🔲 🐕 你是Discord新手吗? 请查看我们的discord 101以获得最佳实践 👉 https://github.com/huggingface/deep-rl-class/blob/main/DISCORD.Md